In [33]:
# To do list:
# Pick H-32324
# VDysDir = Create ./VDysDir
# Search ImagesDirectory for H-12313
# find Case#
# find files .nii.gz > then pick the first! [but print warning]
# copy image to VDysDir
# List all nrrd
# Find ones with label > if several print warn > Pick last
# Copy nrrd to VDysDir
# List of segments = ['ZUNU_vida-sublobes.hdr', ...]
# Find sublobes and recenter them, and put them in VDysDir

In [1]:
# Find H-Number
import re
VidaDirectory = r"Y:\E-Kelly-VidaSegmentals&Sublobes\3- 20Spiromics-10ForKelly-ForERS23\Case53-IA210273_H-17727_20160125_141016_20230524_121005_export"
pattern = r'H-\d{5}'  # Regular expression pattern
matches = re.findall(pattern, VidaDirectory)  # Find all matches
HNUM = matches[0]  # Print the matched patterns
print(HNUM)

H-17727


In [3]:
# Create VDysDir Dir
import os
VDysDir = os.path.join(VidaDirectory, 'VDysDir')
if not os.path.exists(VDysDir):
        os.makedirs(VDysDir)
print("VDysDir created in the VIDA folder")

VDysDir created in the VIDA folder


In [6]:
# Find Case# by seraching in the done folder trees
ImagesDirectory = r"Y:\D-Images\SPIROMICS-SubStudy\2-Results-CheckedDoneTemp\1-Done"
def find_folder_with_hnum(path, HNUM):
    for root, dirs, files in os.walk(path):
        if root[-12:] == '1-Duplicates': continue
        for file in files:
            if HNUM in file:
                while True:
                    if "Case-" in os.path.basename(root):
                        return os.path.basename(root)
                    root = os.path.dirname(root)
                    if root == path or root == os.path.dirname(root):
                        break
    return None
result = find_folder_with_hnum(ImagesDirectory, HNUM)
if result: print("Found folder:", result)
else: print("No matching folder found.")
pattern = r'Case-\d{2}'  # Regular expression pattern
matches = re.findall(pattern, result)  # Find all matches
CNUM = matches[0]  # Print the matched patterns
print(CNUM)

Found folder: Case-53-Spiromics-46195695
Case-53


In [9]:
import glob
# Use glob to find all files with the extension ".nii.gz"
CASEPATH = os.path.join(ImagesDirectory, result)
IMGPATH = glob.glob(CASEPATH + '/**/*.nii.gz', recursive=True)
# Print the list of files
for IMPTH in IMGPATH:
    print(os.path.basename(IMPTH))
print('\nPicked the first as the target image')

H-17727_IA210273_SPI-IA210273-PBV_EXPIRATION__0.5__QR40__3__A_80KV_Qr40d_46195695.nii.gz
H-17727_IA210273_SPI-IA210273-PBV_EXPIRATION__0.75__QR40__A_80KV_Qr40d_46194659.nii.gz

Picked the first as the target image


In [31]:
import shutil
source_file = IMGPATH[0]
destination_folder = VDysDir
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)
print('Copied that image to VDysDir')

Copied image to VDysDir


In [29]:
# Use glob to find all files with the extension ".nrrd"
file_list = glob.glob(CASEPATH + '/**/*.nrrd', recursive=True)
# Filter the file list based on file names containing "Final" and "label"
LABELS = [file for file in file_list if 'Final' in file and 'label' in file]
for LABEL in LABELS:
    print(os.path.basename(LABEL))
LABELPATH = LABELS[-1]
print('\nPicked the last as the target labelmap')

S-Final-1-label.nrrd

Picked the last


In [32]:
source_file = LABELPATH
destination_folder = VDysDir
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)
print('Copied that labelmap to VDysDir')

Copied label to VDysDir


In [14]:
SEGMENTS = ['ZUNU_vida-vessels.hdr', 'ZUNU_vida-sublobes.hdr', 'ZUNU_vida-lung.hdr', 'ZUNU_vida-airtree.hdr']
# re-center vessels and lung masks:
import SimpleITK as sitk
import numpy as np
imfn = os.path.join(VDysDir, os.path.basename(IMGPATH[0]))
iim = sitk.ReadImage(imfn)
for SEG in SEGMENTS:
    segfn = os.path.join(VidaDirectory, SEG)
    segim = sitk.ReadImage(segfn)
    snda = sitk.GetArrayFromImage(segim)
    snds = np.flip(snda, axis=0)
    fseg = sitk.GetImageFromArray(snds)
    fseg.CopyInformation(iim)
    outfn = SEG[0:-4] + '-recentered.nii.gz'
    sitk.WriteImage(fseg, os.path.join(VDysDir, outfn)) # flipped
    print('saved:', outfn)

saved: ZUNU_vida-vessels-recentered.nii.gz
saved: ZUNU_vida-sublobes-recentered.nii.gz
saved: ZUNU_vida-lung-recentered.nii.gz
saved: ZUNU_vida-airtree-recentered.nii.gz
